In [ ]:
import pandas as pd
import numpy as np

file = "../data/instagram.csv"

df = pd.read_csv(file, low_memory=False, parse_dates=[0])

df.head()

In [ ]:
# increase pandas default max_rows so I can view all columns at once
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [ ]:
df.head(2)

In [ ]:
# There are features without values in them
# Here are randomly chosen variables without values

In [ ]:
df["Card Teaser Clicks"].isnull().sum()

In [ ]:
df["Poll Votes"].isnull().sum()

In [ ]:
df["Follow Video Views"].isnull().sum()

In [ ]:
df["Negative Feedback"].isnull().sum()

In [ ]:
df["Profile Clicks"].isnull().sum()

In [ ]:
df.shape  # Number of rows and columns before data cleaning

In [ ]:
list_features = list(df.columns)  # A list of the column names

In [ ]:
list_features

In [ ]:
len(list_features)  # Total number of features before data cleaning

In [ ]:
%cd ..

In [ ]:
# Data Cleaning
from src.clean_data import clean_df

In [ ]:
insta_df = clean_df(df)
insta_df.head()

In [ ]:
insta_df.shape  # Number of rows and columns after data cleaning

In [ ]:
insta_df.columns

In [ ]:
features_after_cleaning = list(insta_df.columns)  # list of features after data cleaning

In [ ]:
len(features_after_cleaning)  # Number of features after data cleaning

In [ ]:
len(list_features) - len(
    features_after_cleaning
)  # 127 features did not have values in them

In [ ]:
insta_df.info()  # A conscise summary of the cleaned dataframe

In [ ]:
# I need to convert the data types to float
insta_df[["Engagement Rate (per Impression)", "Engagement Rate (per Reach)"]].tail()

In [ ]:
# Remove the '%' sign
insta_df["Engagement Rate (per Impression)"] = insta_df[
    "Engagement Rate (per Impression)"
].str.replace("%", "")

# Remove the comma ','
insta_df["Engagement Rate (per Impression)"] = insta_df[
    "Engagement Rate (per Impression)"
].str.replace(",", "")


# Convert to float
insta_df["Engagement Rate (per Impression)"] = insta_df[
    "Engagement Rate (per Impression)"
].astype(float)

In [ ]:
# Remove the '%' sign
insta_df["Engagement Rate (per Reach)"] = insta_df[
    "Engagement Rate (per Reach)"
].str.replace("%", "")

# Remove the comma ',''
insta_df["Engagement Rate (per Reach)"] = insta_df[
    "Engagement Rate (per Reach)"
].str.replace(",", "")

# Convert to float
insta_df["Engagement Rate (per Reach)"] = insta_df[
    "Engagement Rate (per Reach)"
].astype(float)

In [ ]:
insta_df.info()  # Confirm the data type change

## Analysis and Visualization

### Impressions and Reach

The following sub-sections provide insights on the Impressions, Reach, and Engagements of the Instagram posts.

#### QUESTION: What type of content generates the most impressions?

In [ ]:
# create a slice of the dataframe containing non-zero values for the `Impression` column.
valid_impressions = insta_df[
    (insta_df["Impressions"] != 0) & ~insta_df["Impressions"].isna()
]

# group by `Content Type` and calculate the mean `Impressions for each content type
content_df = (
    valid_impressions.groupby("Content Type")["Impressions"]
    .mean()
    .reset_index()
    .sort_values("Impressions", ascending=False)
)

In [ ]:
# import necessary plotting libraries
from bokeh.io import output_notebook

output_notebook()

In [ ]:
# create a bar chart showing the average impressions by content type
from bokeh.plotting import figure, output_file, show

x_range = content_df["Content Type"]
y_range = content_df["Impressions"]

p = figure(
    title="Average Impressions by Content Type",
    x_range=x_range,
    height=400,
    width=600,
    y_axis_label="Average Impression",
)

p.vbar(x=x_range, top=y_range, width=0.8)

p.yaxis.minor_tick_out = 0
p.y_range.start = 1000

show(p)

#### QUESTION: What is the trend of impressions over the years?

In [ ]:
# import relevant plotting libraries
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
# I reseted the index and renamed it to date
valid_impressions.reset_index(inplace=True)
valid_impressions.rename(columns={"index": "date"}, inplace=True)

In [ ]:
valid_impressions.head(2)

In [ ]:
# Converted the date column to a datetime format
valid_impressions["Date"] = pd.to_datetime(valid_impressions["Date"])

In [ ]:
# Set the date as index
valid_impressions.set_index("Date", inplace=True)

In [ ]:
valid_impressions.head(2)

In [ ]:
# calculate the mean impressions per month
mmi = valid_impressions["Impressions"].resample("M").mean()

# forward fill the data for months with empty values and generate a line plot
ax = mmi.fillna(method="ffill").plot(title="Average Impressions over time")

plt.show();

#### QUESTION: When is the best time to post on Instagram?

In [ ]:
# group the dataframe by content type and time period the post was made
cbt = (
    valid_impressions.groupby(["Content Type", "Time Period"])["Reach"]
    .mean()
    .reset_index()
)

time_period_order = ["morning", "afternoon", "evening"]

# arrange the time period in order of morning, afternoon, and evening.
cbt["Time Period"] = pd.Categorical(
    cbt["Time Period"], categories=time_period_order, ordered=True
)

# sort the grouped dataframe by time period order
cbt = cbt.sort_values(["Content Type", "Time Period"])
cbt.reset_index(drop=True, inplace=True)

In [ ]:
# The unique contents
cbt["Content Type"].unique()

In [ ]:
# slice the grouped dataframe into content types
carousel = cbt.iloc[:3]
carousel.name = "Carousel"

photo = cbt.iloc[3:6]
photo.name = "Photos"

video = cbt.iloc[6:9]
video.name = "Videos"

content = (carousel, photo, video)

In [ ]:
# create a function that plots a Bokeh bar chart for the time period data.
def plot_bar(df):
    p = figure(title=df.name, x_range=list(df["Time Period"]), height=300, width=400)

    p.vbar(x="Time Period", top="Reach", color="#99CCFF", source=df, width=0.8)

    p.axis.major_tick_out = 0

    p.xgrid.grid_line_color = None

    p.yaxis.axis_label = "Average Reach"
    p.yaxis.minor_tick_out = 0
    p.y_range.start = 1000
    p.y_range.end = 2500

    return p

In [ ]:
# import the gridplot function to display the plots in a gridplot
from bokeh.layouts import gridplot

# create an empty list and append the bar plots in list
plots = []
for c in content:
    plot = plot_bar(c)
    plots.append(plot)

# create a 2x2 layout for the plots and display it
layout = gridplot(plots, ncols=2, toolbar_location=None)
show(layout)

#### QUESTION: What type of content has the broadest appeal?

In [ ]:
# group sliced dataframe by content type and calaculate the mean "Organic Reach"
org_reach = (
    valid_impressions.groupby("Content Type")["Organic Reach"].mean().reset_index()
)

# sort the grouped in descending order
org_reach.sort_values("Organic Reach", ascending=False, inplace=True)

In [ ]:
# create a bar chart showing the average organic reach by content type
from bokeh.plotting import figure, show

x_range = org_reach["Content Type"]
y_range = org_reach["Organic Reach"]

p = figure(
    title="Average organic reach of content",
    x_range=x_range,
    height=400,
    width=600,
    y_axis_label="Average Organic Reach",
)

p.vbar(x=x_range, top=y_range, width=0.8)

p.yaxis.minor_tick_out = 0
p.y_range.start = 1000

show(p)

#### Question: What type of contents generate the most authentic engagements?

In [ ]:
# group sliced dataframe by content type and calaculate the mean "Organic Reach"
org_reach = (
    valid_impressions.groupby("Content Type")["Organic Impressions"]
    .mean()
    .reset_index()
)

# sort the grouped in descending order
org_reach.sort_values("Organic Impressions", ascending=False, inplace=True)

In [ ]:
# create a bar chart showing the average organic reach by content type
from bokeh.plotting import figure, show

x_range = org_reach["Content Type"]
y_range = org_reach["Organic Impressions"]

p = figure(
    title="Average organic impressions of content",
    x_range=x_range,
    height=400,
    width=600,
    y_axis_label="Average Organic Impression",
)

p.vbar(x=x_range, top=y_range, width=0.8)

p.yaxis.minor_tick_out = 0
p.y_range.start = 1000

show(p)